In [1]:
import pandas as pd
import itertools
from sklearn.linear_model import Ridge
from dateutil.relativedelta import relativedelta
from sklearn.metrics import mean_absolute_error
import numpy as np
%matplotlib inline

In [3]:
data = pd.read_csv('/dsg/demand_anonymized_20170802.csv', delimiter=';', parse_dates=["Month"])
eval = pd.read_csv('/dsg/eval.csv')

In [4]:
# create aggregated by month series
series = data.groupby(["SalOrg", "Material", "Month"]).agg({ "OrderQty":"sum", 
                                                            "PRICE": "median", 
                                                            "Name_Of_Competitor":"mean",
                                                           "COMP_PRICE_MIN":"mean",
                                                           "COMP_PRICE_AVG":"mean",
                                                           "COMP_PRICE_MAX":"mean" }).reset_index()
series = series.sort_values(by=["SalOrg", "Material", "Month"])
series['PRICE_DIFF'] = series['PRICE'] - series['COMP_PRICE_AVG']
series.head()

,SalOrg,Material,Month,COMP_PRICE_AVG,OrderQty,COMP_PRICE_MAX,COMP_PRICE_MIN,Name_Of_Competitor,PRICE,PRICE_DIFF
0,97LK,00IYcj,2012-05-01,170.1,2,170.1,170.1,1.0,137.7,-32.4
1,97LK,00IYcj,2012-06-01,170.1,13,170.1,170.1,1.0,137.7,-32.4
2,97LK,00IYcj,2012-07-01,170.1,1,170.1,170.1,1.0,137.7,-32.4
3,97LK,00IYcj,2012-09-01,170.1,30,170.1,170.1,1.0,137.7,-32.4
4,97LK,00IYcj,2012-11-01,170.1,1,170.1,170.1,1.0,137.7,-32.4


In [5]:
# fill empty
eval_comb = eval[['Material', 'SalOrg']]
eval_comb = list(set([tuple(x) for x in eval_comb.values]))

comb = list(itertools.product(*[eval_comb, list(series['Month'].unique())]))
comb = [(t[0], t[1], m) for t, m in comb]

series2 = pd.DataFrame(comb, columns=['Material', 'SalOrg', 'Month'])
series2 = series2.sort_values(by=['Material', 'SalOrg', 'Month' ])
series2 = series2.merge(series, on=['Month', 'Material', 'SalOrg'], how='left')
series2['PRICE_DIFF'] = series2['PRICE_DIFF'].fillna(series2['PRICE_DIFF'].mean())
series2['Name_Of_Competitor'] = series2['Name_Of_Competitor'].fillna(series2['Name_Of_Competitor'].mean())
series2['COMP_PRICE_MIN'] = series2['COMP_PRICE_MIN'].fillna(series2['COMP_PRICE_MIN'].mean())
series2['COMP_PRICE_AVG'] = series2['COMP_PRICE_AVG'].fillna(series2['COMP_PRICE_AVG'].mean())
series2['COMP_PRICE_MAX'] = series2['COMP_PRICE_MAX'].fillna(series2['COMP_PRICE_MAX'].mean())
series2['PRICE'] = series2['PRICE'].fillna(series2['PRICE'].mean())
series2['OrderQty'] = series2['OrderQty'].fillna(0)

In [6]:
series2['OrderQtyLog'] = np.log1p(series2['OrderQty'])

In [7]:
# create lagged features
def lag_feature(df, colname, lag, adv):
    cols_lagged = []
    for i in range(lag, 0, -1):
        col_lagged = '{}(t-{})'.format(colname, i)
        df[col_lagged] = df.groupby(by=['SalOrg', 'Material'])[colname].shift(i)
        cols_lagged.append(col_lagged)
    cols_adv = []
    for i in range(1, adv):
        col_adv = '{}(t+{})'.format(colname, i)
        df[col_adv] = df.groupby(by=['SalOrg', 'Material'])[colname].shift(-i)
        cols_adv.append(col_adv) 
        
    return df, cols_lagged, cols_adv

series, lagged_pr, adv = lag_feature(series2, 'OrderQtyLog', 12, 3)
series, lagged_1, _ = lag_feature(series, 'COMP_PRICE_MIN', 12, 0)
series, lagged_2, _ = lag_feature(series, 'PRICE_DIFF', 12, 0)
series, lagged_3, _ = lag_feature(series, 'COMP_PRICE_AVG', 12, 0)
series, lagged_4, _ = lag_feature(series, 'COMP_PRICE_MAX', 12, 0)
lagged = lagged_pr + lagged_1 + lagged_2 + lagged_3 + lagged_4
series.head()

,Material,SalOrg,Month,COMP_PRICE_AVG,OrderQty,COMP_PRICE_MAX,COMP_PRICE_MIN,Name_Of_Competitor,PRICE,PRICE_DIFF,...,COMP_PRICE_MAX(t-10),COMP_PRICE_MAX(t-9),COMP_PRICE_MAX(t-8),COMP_PRICE_MAX(t-7),COMP_PRICE_MAX(t-6),COMP_PRICE_MAX(t-5),COMP_PRICE_MAX(t-4),COMP_PRICE_MAX(t-3),COMP_PRICE_MAX(t-2),COMP_PRICE_MAX(t-1)
0,00GB1f,yqSu,2012-01-01,239.210419,0.0,245.554231,233.012279,0.390422,258.79165,-2.350168,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00GB1f,yqSu,2012-02-01,239.210419,0.0,245.554231,233.012279,0.390422,258.79165,-2.350168,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,245.554231
2,00GB1f,yqSu,2012-03-01,239.210419,1.0,245.554231,233.012279,0.000000,258.79165,-2.350168,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,245.554231,245.554231
3,00GB1f,yqSu,2012-04-01,239.210419,3.0,245.554231,233.012279,0.000000,258.79165,-2.350168,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,245.554231,245.554231,245.554231
4,00GB1f,yqSu,2012-05-01,239.210419,2.0,245.554231,233.012279,0.000000,258.79165,-2.350168,...,NaN,NaN,NaN,NaN,NaN,NaN,245.554231,245.554231,245.554231,245.554231


In [8]:
def create_agg_features(data, columns, name):
    temp = data.loc[(data["Month"] > pd.to_datetime('2014-09-01'))&
                    (data["Month"] < pd.to_datetime('2016-09-01'))].groupby(columns)["OrderQty"].agg(["median", "mean", "std", "min", "max"]).reset_index()
    new_cols = [col + name for col in ["median", "mean", "std", "min", "max"]]
    temp.columns = columns + new_cols
    return temp, new_cols

sm_temp, sm_cols = create_agg_features(series, ["SalOrg", "Material"], "_s_m_")
series = series.merge(sm_temp, how='left', on=["SalOrg", "Material"])

s_temp, s_cols = create_agg_features(series, ["SalOrg"], "_s_")
series = series.merge(s_temp, how='left', on=["SalOrg"])

m_temp, m_cols = create_agg_features(series, ["Material"], "_m_")
series = series.merge(m_temp, how='left', on=["Material"])

series.head()

,Material,SalOrg,Month,COMP_PRICE_AVG,OrderQty,COMP_PRICE_MAX,COMP_PRICE_MIN,Name_Of_Competitor,PRICE,PRICE_DIFF,...,median_s_,mean_s_,std_s_,min_s_,max_s_,median_m_,mean_m_,std_m_,min_m_,max_m_
0,00GB1f,yqSu,2012-01-01,239.210419,0.0,245.554231,233.012279,0.390422,258.79165,-2.350168,...,0.0,25.389641,202.069177,0.0,13540.0,0.0,0.26087,0.448978,0.0,1.0
1,00GB1f,yqSu,2012-02-01,239.210419,0.0,245.554231,233.012279,0.390422,258.79165,-2.350168,...,0.0,25.389641,202.069177,0.0,13540.0,0.0,0.26087,0.448978,0.0,1.0
2,00GB1f,yqSu,2012-03-01,239.210419,1.0,245.554231,233.012279,0.000000,258.79165,-2.350168,...,0.0,25.389641,202.069177,0.0,13540.0,0.0,0.26087,0.448978,0.0,1.0
3,00GB1f,yqSu,2012-04-01,239.210419,3.0,245.554231,233.012279,0.000000,258.79165,-2.350168,...,0.0,25.389641,202.069177,0.0,13540.0,0.0,0.26087,0.448978,0.0,1.0
4,00GB1f,yqSu,2012-05-01,239.210419,2.0,245.554231,233.012279,0.000000,258.79165,-2.350168,...,0.0,25.389641,202.069177,0.0,13540.0,0.0,0.26087,0.448978,0.0,1.0


In [9]:
# validation folds
def create_validation(data, date, m):
    return data[(data["Month"] > pd.to_datetime(date) - relativedelta(months=m))&
                (data["Month"] < pd.to_datetime(date))].index, data[(data["Month"] >= pd.to_datetime(date)) & 
                (data["Month"] < pd.to_datetime(date) + relativedelta(months=3))].index

validation_months = ['2016-09-01', '2016-10-01', '2016-11-01']

folds = []
for month in validation_months:
    fold = create_validation(series, month, 3)
    folds.append(fold)

In [10]:
all = len(series.groupby(by=['SalOrg', 'Material']))

In [18]:
errors = {}
all_errors = []
features = lagged + sm_cols + m_cols + s_cols

for fold in folds:
    fold_errors = []
    
    processed = 1
    for index, group in series.groupby(by=['SalOrg', 'Material']):
        if index not in errors:
            errors[index] = []
        
        train_s = group.loc[group.index.intersection(fold[0])]
        test_s = group.loc[group.index.intersection(fold[1])]
        
        train_y = train_s[['OrderQtyLog'] + adv]
        train_x = train_s[features]

        test_y = test_s[['OrderQtyLog'] + adv]
        test_x = test_s[features]

        model = Ridge(alpha=12)
        model.fit(train_x, train_y)

        test_y_predicted = model.predict(test_x)
        test_y_predicted[test_y_predicted < 0] = 0

        error = mean_absolute_error(np.expm1(test_y), np.expm1(test_y_predicted))
        
        errors[index].append(error)
        all_errors.append(error)
        fold_errors.append(error)
        
        processed += 1
        if(processed % 1000 == 0):
            print('Trained {}/{} models, mean fold error: {}'.format(processed, all, np.mean(fold_errors)))
        
    print('Mean fold error: {}'.format(np.mean(fold_errors)))
    
np.mean(all_errors)

Trained 1000/38676 models, mean fold error: 14.840949474542642
Trained 2000/38676 models, mean fold error: 15.860454165722533
Trained 3000/38676 models, mean fold error: 13.169781685821365
Trained 4000/38676 models, mean fold error: 13.759725506158336
Trained 5000/38676 models, mean fold error: 13.7078871875492
Trained 6000/38676 models, mean fold error: 14.08547982413439
Trained 7000/38676 models, mean fold error: 13.1866940895679
Trained 8000/38676 models, mean fold error: 14.142615202730045
Trained 9000/38676 models, mean fold error: 13.803076989006945
Trained 10000/38676 models, mean fold error: 13.51636634987881
Trained 11000/38676 models, mean fold error: 13.31841267430782
Trained 12000/38676 models, mean fold error: 12.8484524220572
Trained 13000/38676 models, mean fold error: 12.271904840395415
Trained 14000/38676 models, mean fold error: 11.988172455526682
Trained 15000/38676 models, mean fold error: 11.957895086290794
Trained 16000/38676 models, mean fold error: 11.6860047123

6318.0355736997835

In [19]:
models = {}
for index, group in series.groupby(by=['SalOrg', 'Material']):
    train = group.loc[group.index.intersection(fold[1])]

    train_y = train[['OrderQtyLog'] + adv]
    train_x = train[features]

    model = Ridge(alpha=8)
    model.fit(train_x, train_y)
    models[index] = model

In [ ]:
lagged_final = lagged
lagged_final.append('OrderQtyLog')
lagged_final.pop(0)
fetures_final = lagged_final + sm_cols + m_cols + s_cols
fetures_final

In [ ]:
test_i = series[series["Month"]==pd.to_datetime('2017-03-01')].index

predictions = []
for index, group in series.groupby(by=['SalOrg', 'Material']):
    model = models[index]
    test = group.loc[group.index.intersection(test_i)]
    test_x = series.loc[test_i][fetures_final]
    test_prediction = model.predict(test_x)
    prediction = np.expm1(test_prediction)
    row = list(index) + list(prediction)
    predictions.append(row)
predictions

In [ ]:
pre_df = pd.DataFrame(predictions, columns=['SalOrg', 'Material', 'OrderQty(t+1)', 'OrderQty(t+2)','OrderQty(t+3)'])
ml = pd.melt(pre_df, id_vars=['Material', 'SalOrg'], value_vars=['OrderQty(t+1)', 'OrderQty(t+2)','OrderQty(t+3)'])
ml['date'] = ml.variable.replace({'OrderQty(t+1)':'2017-04', 'OrderQty(t+2)':'2017-05', 'OrderQty(t+3)':'2017-06'})
ml.head()

In [ ]:
result = eval.merge(ml, on=['Material', 'SalOrg', 'date'])
result

In [ ]:
result['demand'] = result['value']
result[['ID', 'demand']].to_csv('/dsg/linear_models_11.98.csv', index=False)